In [14]:
import keras
import pandas as pd
from keras import layers, Sequential
import argparse
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import os
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.optimizers import Adam, SGD, RMSprop

In [27]:
df = pd.read_csv('gesture_data.csv')
x = df.copy()
y = x.pop('class')
y, class_list = y.factorize()
x = x.astype('float64')
y = keras.utils.to_categorical(y)

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2,
                                                    random_state=0)

In [16]:
Model_Checkpoint_Callback = ModelCheckpoint('gesture_model.h5', 
                                            monitor='val_accuracy', 
                                            verbose=1, 
                                            save_best_only=True, 
                                            mode='max',
                                            save_freq = "epoch")
Early_Stop = EarlyStopping(monitor='val_accuracy',
                              patience=10)

In [21]:
model = Sequential()
model.add(Input(shape=(92)))
model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(6, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train,y_train,
                    batch_size=16,
                    epochs=100,
                    validation_data=(x_test, y_test),
                    callbacks=[Model_Checkpoint_Callback,Early_Stop])

Epoch 1/100
 94/102 [==========================>...] - ETA: 0s - loss: 0.8609 - accuracy: 0.6908
Epoch 1: val_accuracy improved from -inf to 0.86946, saving model to gesture_model.h5
102/102 [==============================] - 2s 10ms/step - loss: 0.8289 - accuracy: 0.7024 - val_loss: 0.3313 - val_accuracy: 0.8695
Epoch 2/100
 30/102 [=======>......................] - ETA: 0s - loss: 0.3151 - accuracy: 0.8833

C:\Users\Tommy\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 89/102 [=========================>....] - ETA: 0s - loss: 0.2436 - accuracy: 0.9136
Epoch 2: val_accuracy improved from 0.86946 to 0.93350, saving model to gesture_model.h5
102/102 [==============================] - 1s 6ms/step - loss: 0.2403 - accuracy: 0.9150 - val_loss: 0.1824 - val_accuracy: 0.9335
Epoch 3/100
 88/102 [========================>.....] - ETA: 0s - loss: 0.1488 - accuracy: 0.9574
Epoch 3: val_accuracy improved from 0.93350 to 0.95320, saving model to gesture_model.h5
102/102 [==============================] - 1s 5ms/step - loss: 0.1401 - accuracy: 0.9593 - val_loss: 0.1071 - val_accuracy: 0.9532
Epoch 4/100
 93/102 [==========================>...] - ETA: 0s - loss: 0.0864 - accuracy: 0.9738
Epoch 4: val_accuracy improved from 0.95320 to 0.96552, saving model to gesture_model.h5
102/102 [==============================] - 1s 8ms/step - loss: 0.0829 - accuracy: 0.9754 - val_loss: 0.0750 - val_accuracy: 0.9655
Epoch 5/100
101/102 [============================>.] - ETA: 0

In [23]:
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import load_model
import numpy as np
model_test = load_model("gesture_model.h5")
y_pred = model_test.predict(x_test)
y_pred = np.argmax(y_pred, axis = 1)
y_test=np.argmax(y_test, axis=1)
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

13/13 [==============================] - 0s 3ms/step
Confusion Matrix
[[50  0  0  0  0  0]
 [ 0 78  0  0  0  0]
 [ 0  0 53  0  0  0]
 [ 0  0  0 87  0  0]
 [ 0  0  0  0 58  0]
 [ 0  0  0  0  0 80]]


In [25]:
print('Classification Report')
print(classification_report(y_test, y_pred, target_names=class_list))

Classification Report
                  precision    recall  f1-score   support

   crossing_arms       1.00      1.00      1.00        50
crossing_fingers       1.00      1.00      1.00        78
  netural&others       1.00      1.00      1.00        53
  touching_faces       1.00      1.00      1.00        87
    touching_jaw       1.00      1.00      1.00        58
   touching_neck       1.00      1.00      1.00        80

        accuracy                           1.00       406
       macro avg       1.00      1.00      1.00       406
    weighted avg       1.00      1.00      1.00       406

